In [1]:
from osgeo import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm.auto import trange

d:\Anaconda\envs\dem\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Coordinate transformation

In [ ]:
# xy->XY
def xy2XY(x, y, geo):
    X = geo[0] + x * geo[1] + y * geo[2]
    Y = geo[3] + x * geo[4] + y * geo[5]
    return X, Y


def xy2XY_2(xy, geo):
    X = geo[0] + xy[0] * geo[1] + xy[1] * geo[2]
    Y = geo[3] + xy[0] * geo[4] + xy[1] * geo[5]
    return X, Y


# XY->xy
def XY2xy(X, Y, geo):
    a = np.array([[geo[1], geo[2]], [geo[4], geo[5]]])
    b = np.array([X - geo[0], Y - geo[3]])
    x, y = np.linalg.solve(a, b)
    return round(x), round(y)


def XY2xy_2(X, Y, geo):
    a = np.array([[geo[1], geo[2]], [geo[4], geo[5]]])
    b = np.array([X - geo[0], Y - geo[3]])
    x, y = np.linalg.solve(a, b)
    return x, y

#### Obtain the HCP index and error within the DEM range

In [ ]:
def Select_CP_inDEM(IMG_geo, IMG_row, IMG_col, dem_data,CP_X, CP_Y,CP_Z):
    CP_select_index = []
    CP_select_error=[]

    for i in range(0, len(CP_X)):
        temp_c, temp_r = XY2xy(CP_X[i], CP_Y[i], IMG_geo)
        if temp_r >= 0 and temp_r < IMG_row and temp_c >= 0 and temp_c < IMG_col:
            if dem_data[temp_r, temp_c] != -222:
                CP_select_index.append(i)
                CP_select_error.append(dem_data[temp_r, temp_c] - CP_Z[i])
    
    CP_select_error=np.array(CP_select_error)
    CP_select_index=np.array(CP_select_index)

    return CP_select_index,CP_select_error

#### Analyze the elevation error

In [4]:
def calculate_height_error_SIMPLE(h_error):
    if len(h_error)!=0:
        abs_error = []
        error2 = []
        for i in range(0, len(h_error)):
            abs_error.append(abs(h_error[i]))
            error2.append(pow(h_error[i], 2))
        abs_error.sort()
        mean = sum(h_error) / len(error2)
        mae = sum(abs_error) / len(error2)
        rmse = pow((sum(error2) / len(error2)), 0.5)
    else:
        mean = -1
        mae = -1
        rmse = -1
    return mean,mae, rmse

### Set input and output

In [ ]:
# The number of pairs of images to be adjusted
IMG_pair_num = 1

# DEM data path
DEM_data_Path = "dataset/dem/"

# Adjustment methods
PA1_Method_Name = "LAD"

# Elevation error grid size
Grid_Size = 101 # Take into account 5,9 and 17
MAX_Grid_Size = 101


In [ ]:
# Input
# DEM size
ROWs = np.load("dataset/dem_info/ROWs.npy")
COLs = np.load("dataset/dem_info/COLs.npy")
GEOs = np.load("dataset/PA1/GEOs_"+PA1_Method_Name+".npy")
Distance = np.load("dataset/PA1/Distance_"+PA1_Method_Name+".npy")

# HCPs
ControlP = pd.read_excel("dataset/CP/GDCPs_8f3c.xlsx")
ControlP = ControlP.to_numpy()

CPX = np.array(ControlP[:, 0])
CPY = np.array(ControlP[:, 1])
CPZ = np.array(ControlP[:, -1])
print(CPX.shape)

(15285,)


In [ ]:
# Initial Herr Grid
Initial_Herr_Grid_MEAN_1 = np.zeros((IMG_pair_num, Grid_Size, Grid_Size))
Initial_Herr_Grid_MAE_1 = np.zeros((IMG_pair_num,  Grid_Size, Grid_Size))
Initial_Herr_Grid_RMSE_1 = np.zeros((IMG_pair_num,  Grid_Size, Grid_Size))

Initial_Herr_Grid_MEAN_all = np.zeros((IMG_pair_num, Grid_Size, Grid_Size))
Initial_Herr_Grid_MAE_all = np.zeros((IMG_pair_num,  Grid_Size, Grid_Size))
Initial_Herr_Grid_RMSE_all = np.zeros((IMG_pair_num,  Grid_Size, Grid_Size))

# Output name
col1 = "CP_index"
col2 = "CP_error"


### Calculate the elevation error grid

In [ ]:
for Select_IMGpair in range(0, IMG_pair_num):
# for Select_IMGpair in range(0, 1):
    print(" ")
    print("The {} pair".format(Select_IMGpair))

    # Read DEM data
    DEM_data_1 = np.load(DEM_data_Path + str(Select_IMGpair * 2) + ".npy")
    DEM_data_2 = np.load(DEM_data_Path + str(Select_IMGpair * 2 + 1) + ".npy")
    temp_geo1 = np.copy(GEOs[Select_IMGpair * 2])
    temp_geo2 = np.copy(GEOs[Select_IMGpair * 2 + 1])
    temp_distance = np.copy(Distance[Select_IMGpair])

    '''
    Automatically select the appropriate maximum search range
    at least one pixel each time
    '''
    if abs(temp_distance[0]) < abs((MAX_Grid_Size - 1) * temp_geo1[1]):
        temp_distance[0] = (MAX_Grid_Size - 1) * temp_geo1[1]
    if abs(temp_distance[1]) < abs((MAX_Grid_Size - 1) * temp_geo1[5]):
        temp_distance[1] = (MAX_Grid_Size - 1) * temp_geo1[5]
    print("The actual size of the search range{}".format(temp_distance))

    # Calculate the grid step size
    X_step = temp_distance[0] / (Grid_Size - 1)  # The top of Grid_Size needs to be removed
    Y_step = temp_distance[1] / (Grid_Size - 1)

    # Initialize GEO to the corner of the grid
    temp_geo1[0] = temp_geo1[0] + X_step * (Grid_Size - 1) / 2
    temp_geo2[0] = temp_geo2[0] + X_step * (Grid_Size - 1) / 2
    temp_geo1[3] = temp_geo1[3] + Y_step * (Grid_Size - 1) / 2
    temp_geo2[3] = temp_geo2[3] + Y_step * (Grid_Size - 1) / 2

    # Grid search
    for x_index in trange(0, Grid_Size):
        for y_index in range(0, Grid_Size):
            # Screen the HCP within the DEM range and calculate the height difference
            CP_index_1, CP_error_1 = Select_CP_inDEM(
                temp_geo1,
                ROWs[Select_IMGpair * 2],
                COLs[Select_IMGpair * 2],
                DEM_data_1,
                CPX,CPY,CPZ,
            )
            CP_index_2, CP_error_2 = Select_CP_inDEM(
                temp_geo2,
                ROWs[Select_IMGpair * 2+1],
                COLs[Select_IMGpair * 2+1],
                DEM_data_2,
                CPX,CPY,CPZ,
            )
 
            # Analyze the initial height difference
            mean_t, mae_t,rmse_t=calculate_height_error_SIMPLE(CP_error_1)
            Initial_Herr_Grid_MEAN_1[Select_IMGpair,x_index,y_index] =mean_t
            Initial_Herr_Grid_MAE_1[Select_IMGpair,x_index,y_index] =mae_t
            Initial_Herr_Grid_RMSE_1[Select_IMGpair,x_index,y_index] =rmse_t

            # Update GEO
            temp_geo1[3]=temp_geo1[3]-Y_step
            temp_geo2[3]=temp_geo2[3]-Y_step
        temp_geo1[0]=temp_geo1[0]-X_step
        temp_geo2[0]=temp_geo2[0]-X_step
        temp_geo1[3]=temp_geo1[3]+Y_step*(Grid_Size)
        temp_geo2[3]=temp_geo2[3]+Y_step*(Grid_Size)


### Save output

In [ ]:
# Save the search grid
np.save("dataset/PA2/Refer_Herr_Grid/MEAN_1",Initial_Herr_Grid_MEAN_1)
np.save("dataset/PA2/Refer_Herr_Grid/MAE_1",Initial_Herr_Grid_MAE_1)
np.save("dataset/PA2/Refer_Herr_Grid/RMSE_1",Initial_Herr_Grid_RMSE_1)

np.save("dataset/PA2/Refer_Herr_Grid/MEAN_all",Initial_Herr_Grid_MEAN_all)
np.save("dataset/PA2/Refer_Herr_Grid/MAE_all",Initial_Herr_Grid_MAE_all)
np.save("dataset/PA2/Refer_Herr_Grid/RMSE_all",Initial_Herr_Grid_RMSE_all)
